# Assignment - Artificial Intelligence 2

## Assignment Background:

In this assignment you will apply everything covered in the lesson to a new medical imaging problem. You will design, train, and evaluate your own CNN from scratch, making your own decisions about architecture, training strategy, and how to interpret your results.

This assignment uses the **PneumoniaMNIST** dataset, which contains black-and-white chest X-rays with a binary classification task: **Pneumonia** or **No Pneumonia**.

The dataset is split as follows:
- **Training set:** 4,708 images
- **Validation set:** 524 images
- **Test set:** 624 images

The images are **1 channel** (greyscale) and **28×28 pixels**, consistent with the MedMNIST format used in the case study.

This data is made available by *Daniel S. Kermany, Michael Goldbaum, et al., "Identifying medical diagnoses and treatable diseases by image-based deep learning," Cell, vol. 172, no. 5, pp. 1122–1131.e9, 2018.*

### Important!
The code given to you will download the *PneumoniaMNIST* into your CodeSpace workspace. This file is too large to be hosted on GitHub and will cause an error when you try to push your assignment back. Please remember to **delete** the data before you do. If you do accidentally push your assignment back with the data you will need to firstly *delete* the data and then go through your git history and delete its reference there. For specific instructions on how to do this, head to the [handbook](../HB/troubleshoot.html).

## Assignment Questions:

The data loading code is provided below — the design, training, and evaluation are up to you.

The overall aim is to build a CNN that **catches as many pneumonia cases as possible**. In a clinical setting, a missed positive (a patient with pneumonia classified as healthy) is far more serious than a false alarm. This means your primary metric is **recall** (also called sensitivity): the proportion of true pneumonia cases that your model correctly identifies.

That said, a model that simply predicts *pneumonia* for every patient would have perfect recall but be clinically useless. You should therefore also report **precision** (the proportion of positive predictions that are actually correct) to show your model is not just guessing positive every time. The **F1 score** balances the two.

**Your submission should include the following steps:**

**Step 1 — Explore the data**
Plot a sample of images from the training set, with their labels. Check the class balance (how many pneumonia vs. non-pneumonia cases are there?).

**Step 2 — Design a CNN**
Define a CNN architecture suited to this problem. Consider:
- The input has **1 channel** and images are **28×28**
- The output should have **2 classes**
- You may use BatchNorm and Dropout as introduced in the case study

**Step 3 — Train the model**
Train your model using the training and validation loaders. Implement early stopping and a learning rate scheduler as demonstrated in the case study.
Plot the training and validation loss curves.

**Tip:** Try a small number of epochs first to see how long it takes; scale accordingly.

**Step 4 — Evaluate on the test set**
Using the test set, produce:
- A **confusion matrix** (use the function provided)
- An ROC curve with AUC scores
- **Recall**, **precision**, and **F1 score** for each class
    - Use the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function from SKLearn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

from torchvision import transforms
import medmnist
from medmnist import INFO

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED);

In [ ]:
data_flag = 'pneumoniamnist'

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

real_labels = {0: "Normal", 1: "Pneumonia"}

print(f"The learning task is {task}")
print(f"The number of channels is {n_channels}")
print(f"The number of classes is {n_classes}")

In [ ]:
download = True
BATCH_SIZE = 128

DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]) # normalise the data with standard values
])

# load the data
train_loader = torch.utils.data.DataLoader(
    DataClass(split='train', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=BATCH_SIZE, shuffle=True)

val_loader = torch.utils.data.DataLoader(
    DataClass(split='val', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=2*BATCH_SIZE, shuffle=True)

# Test dataset — do not use this until Step 4
test_loader = torch.utils.data.DataLoader(
    DataClass(split='test', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=2*BATCH_SIZE, shuffle=True)

## Step 1 — Explore the data

Plot a sample of images from the training set with their labels. Check the class balance.

In [ ]:
# Plot a sample of training images with labels
images, labels = next(iter(train_loader))

fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i].squeeze(), cmap='gray')
    ax.set_title(real_labels[int(labels[i])], fontsize=8)
    ax.axis('off')
plt.suptitle('Sample training images', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Check class balance — important for a medical classification task
train_labels_all = []
for _, target in train_loader:
    train_labels_all.extend(target.squeeze().tolist())

counts = {real_labels[i]: train_labels_all.count(i) for i in range(n_classes)}
print("Training set class distribution:")
for label, count in counts.items():
    print(f"  {label}: {count} ({100*count/len(train_labels_all):.1f}%)")

## Step 2 — Design a CNN

Define your `CNN` class below. Remember:
- Input: `1` channel, `28×28` images
- Output: `2` classes
- Use the explicit `forward()` pattern from the lesson (call each layer directly, not `nn.Sequential` inside `forward`)

In [ ]:
class CNN_Pneumonia(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        # Block 1: 28x28 → 14x14
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # Block 2: 14x14 → 7x7
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # Block 3: 7x7 → 3x3 (floor(7/2) = 3)
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # Fully connected classifier: 64 * 3 * 3 = 576 input features
        self.fc = nn.Sequential(
            nn.Linear(64 * 3 * 3, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes))

    def forward(self, x):
        # Explicit forward pass
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Confirm the architecture computes the right output shape
model = CNN_Pneumonia(in_channels=n_channels, num_classes=n_classes)
dummy = torch.zeros(1, 1, 28, 28)
print("Output shape:", model(dummy).shape)  # should be [1, 2]
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## Step 3 — Train the model

Train your model using the training and validation loaders. Use early stopping and a learning rate scheduler as shown in the case study. Plot the training and validation loss curves.

In [ ]:
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device("cuda") # Defines the device you want to have data moved to
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f"Using: {device} (Cuda)")

elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using: {device} (Apple Silicon)")

else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

model = CNN_Pneumonia(in_channels=n_channels, num_classes=n_classes).to(device)

In [ ]:
# Hyperparameters
n_epochs    = 20
lr          = 0.001
patience    = 3
epoch_patience = 6

criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.01)
lr_scheduler = ReduceLROnPlateau(optimiser, mode='min', factor=0.1, patience=patience, verbose=True)

best_val_loss  = float('inf')
epochs_no_improve = 0
best_model_params = None

train_losses, val_losses = [], []

for epoch in range(n_epochs):

    # --- Training ---
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.squeeze().long().to(device)
        optimiser.zero_grad()
        loss = criterion(model(data), target)
        loss.backward()
        optimiser.step()
        train_loss += loss.item() * data.size(0)

    # --- Validation ---
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.squeeze().long().to(device)
            output = model(data)
            val_loss += criterion(output, target).item() * data.size(0)
            correct += (output.argmax(1) == target).sum().item()
            total += target.size(0)

    train_loss /= len(train_loader.sampler)
    val_loss  /= len(val_loader.sampler)
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    lr_scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_params = model.state_dict().copy()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == epoch_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {100*correct/total:.1f}%")

# Restore best weights
model.load_state_dict(best_model_params)

In [ ]:
# Plot training and validation loss curves
x_epochs = range(1, len(train_losses) + 1)
plt.plot(x_epochs, train_losses, label='Training loss')
plt.plot(x_epochs, val_losses,  label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(x_epochs)
plt.legend(frameon=False)
plt.title('Training and Validation Loss — PneumoniaMNIST')
plt.show()

## Step 4 — Evaluate on the test set

Collect predictions on the full test set, then produce a confusion matrix, ROC-AUC plot, and report recall, precision, and F1 score per class.

In [ ]:
# Confusion matrix function:

def plot_confusion_matrix(y_true, y_pred, classes=None, normalise=False, colmap=True, ax=None, cmap='inferno'):
    """
    Plot confusion matrix on given axes.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        classes: List of class labels (if None, uses range from data)
        normalise: If True, normalise the confusion matrix
        ax: Matplotlib axes object (if None, creates new figure)
        cmap: Colourmap for the plot
    """
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalise if requested
    if normalise:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Set default classes if not provided
    if classes is None:
        classes = np.unique(y_true)
    
    # Create display
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    # Plot on provided axes or create new figure
    disp.plot(ax=ax, cmap=cmap, colorbar=colmap, values_format='.2f' if normalise else 'd')
    
    if ax is not None:
        ax.set_xlabel('Predicted Label')
        ax.set_ylabel('True Label')
    
    return disp

In [ ]:
# Collect all predictions on the test set — as in the case study (notebook 03)
model.eval()
test_images_all, test_labels_all = [], []
for data, target in test_loader:
    test_images_all.append(data)
    test_labels_all.append(target)

test_images_all = torch.cat(test_images_all).to(device)
test_labels_all = torch.cat(test_labels_all).squeeze()  # [N] — required by sklearn

with torch.no_grad():
    test_output = model(test_images_all)

_, test_preds = torch.max(test_output, 1)

# Move to CPU numpy for sklearn
y_true = test_labels_all.cpu().numpy()
y_pred = test_preds.cpu().numpy()

In [ ]:
# Confusion matrix
fig, ax = plt.subplots(figsize=(17, 8))

plot_confusion_matrix(y_true, y_pred, 
                        classes=[0, 1], 
                        normalise=True,
                        ax=ax)
plt.show()

In [ ]:
# ROC curve
probs = torch.softmax(test_output, dim=1).detach().cpu().numpy()

fig, ax = plt.subplots(figsize=(6, 6))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true == i, probs[:, i])
    auc = roc_auc_score(y_true == i, probs[:, i])
    ax.plot(fpr, tpr, label=f"{real_labels[i]}: AUC = {auc:.2f}")

ax.plot([0, 1], [0, 1], 'k--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve — Test Set')
ax.legend(frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
print(classification_report(y_true, y_pred, target_names=list(real_labels.values())))